In [1]:
import tensorflow as tf

In [2]:
session = tf.InteractiveSession()

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
width = height = 28
flat = width * height # = number of pixels
class_output = 10

In [5]:
# Creating placeholders for input and output

X  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [6]:
# This function will be responsible for generating weights having some noise to prevent vanishing gradient problem.

def weight_variable(shape):
    weight = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weight)


In [7]:
# Function for generating bias variable to avoid "dying" of neurons

def bias_variable(shape):
    bias = tf.constant(0.1, shape=shape)
    return tf.Variable(bias)

In [9]:
"""
Structure of Network
1. Input data
2. Convolution and max pooling
3. Convolution and max pooling
4. Fully connected layer
5. Processing - DropOut
6. Readout layer - Fully Connected
7. Output - Classified digits
"""

'\nStructure of Network\n1. Input data\n2. Convolution and max pooling\n3. Convolution and max pooling\n4. Fully connected layer\n5. Processing - DropOut\n6. Readout layer - Fully Connected\n7. Output - Classified digits\n'

In [10]:
# A function for convolution layers

def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')

In [11]:
# Function for 2x2 max pooling

def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [12]:
# Making first convolution and max pooling layer

# Weights and Biases
W_conv1 = weight_variable([5, 5, 1, 32]) # Size of the filter/kernel: 5x5; Input channels: 1 (greyscale); 32 feature maps
b_conv1 = bias_variable([32]) # need 32 biases for 32 outputs

# Converting images of dataset to tensors
X_image = tf.reshape(X, [-1, 28, 28, 1])

# Convolving the weight tensor and adding bias
convolve1 = conv2d(X_image, W_conv1) + b_conv1

# Appling ReLu activation
h_conv1 = tf.nn.relu(convolve1)

# Applying max pooling
h_pool1 = max_pool_2x2(h_conv1)

layer1 = h_pool1

In [13]:
# Making second convolution and max pooling layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

convolve2 = conv2d(layer1, W_conv2) + b_conv2

h_conv2 = tf.nn.relu(convolve2)

h_pool2 = max_pool_2x2(h_conv2)

layer2 = h_pool2

In [14]:
# Making third layer - Fully connected layer to use softmax and create probabilities in the end.

# Weights and bias
# Last layer produce a feature map of 7x7, number of feature map = 64, output to softmax = 1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# Flattening second layer
layer2_matrix = tf.reshape(layer2, [-1, 7 * 7 * 64])

# Applying weights and bias
matmul_fc1 = tf.matmul(layer2_matrix, W_fc1) + b_fc1

# Applying ReLu
h_fc1 = tf.nn.relu(matmul_fc1)

layer3 = h_fc1

In [15]:
# Droput - regularization

keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

In [16]:
# Final layer - Read out layer (softmax, fully connected)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

matmul_fc2 = tf.matmul(layer3_drop, W_fc2) + b_fc2

# Applying softmax activation
y_conv = tf.nn.softmax(matmul_fc2)
layer4 = y_conv

In [18]:
%timeit

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(layer4), reduction_indices=[1]))

In [19]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [20]:
correct_prediction = tf.equal(tf.argmax(layer4, 1), tf.argmax(y_, 1))

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
%timeit session.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


1 loops, best of 3: 113 ms per loop


In [25]:
for _ in range(1000):
    batch = mnist.train.next_batch(50)
    if _ % 100 == 0:
        train_accuracy = accuracy.eval(session=session, feed_dict={X: batch[0], y_: batch[1], keep_prob: 1.0})
        print "Step %d : %.5f" % (_ , float(train_accuracy))
    train_step.run(session=session, feed_dict={X: batch[0], y_: batch[1], keep_prob: 0.5})

Step 0 : 0.96000
Step 100 : 0.98000
Step 200 : 0.94000
Step 300 : 0.92000
Step 400 : 0.98000
Step 500 : 1.00000
Step 600 : 0.94000
Step 700 : 0.98000
Step 800 : 0.96000
Step 900 : 1.00000


In [26]:
print "test accuracy %g"%accuracy.eval(session=session, feed_dict={X: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

test accuracy 0.9736


In [ ]:
sess